# MMOB Project

# import

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [664]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
data_file ='https://raw.githubusercontent.com/DunodMax/MMOB/main/Dataset.txt'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}
from collections import namedtuple

In [665]:
LPMC.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,18,4,0,0,3,3,6,5,0.0,1,...,0.171389,0.0,0.334444,0.0,0,0.196389,0.0,0.53,0.0,0.035361
1,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.0,0.062222,0.0,0,0.117222,0.0,0.41,0.0,0.097156
2,21,5,1,1,4,3,1,5,0.0,1,...,0.083889,0.0,0.293611,0.0,0,0.167778,0.0,0.46,0.0,0.243377
3,22,6,0,0,3,3,6,1,1.0,1,...,0.109167,0.0,0.133333,0.0,0,0.093889,1.5,0.23,0.0,0.204142
4,31,8,1,5,1,3,1,1,1.0,1,...,0.101111,0.0,0.038056,0.0,0,0.055000,1.5,0.14,0.0,0.075758


# Model 0

all useful columns for this model are used as variables

In [666]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')
female=Variable('female')
age=Variable('age')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [667]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [668]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions

In [669]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

In [670]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [671]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.832004,0.107570,-35.623398,0.0
asc_drive,-1.235561,0.080397,-15.368167,0.0
asc_pt,-0.528693,0.054766,-9.653749,0.0
beta_cost,-0.169781,0.013107,-12.953165,0.0
beta_time,-5.450143,0.202666,-26.892180,0.0


In [672]:
res.compileEstimationResults(all_results)

,Model0
Number of estimated parameters,5.000000
Sample size,5000.000000
Final log likelihood,-4566.446495
Akaike Information Criterion,9142.892989
Bayesian Information Criterion,9175.478955
asc_cycle,-3.832002
asc_drive,-1.235559
asc_pt,-0.528693
beta_cost,-0.169777
beta_time,-5.450140


In [673]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [674]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [675]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [676]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model1_time_specification'
all_results['Model1_time_specification'] = biogeme_model1.estimate()
results_alt_spec_time = biogeme_model1.estimate()
results_alt_spec_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.508888,0.191038,-23.602111,0.0
asc_drive,-1.885804,0.122746,-15.363431,0.0
asc_pt,-2.339783,0.125804,-18.598663,0.0
beta_cost,-0.146075,0.014851,-9.836160,0.0
beta_time_cycle,-6.124165,0.543081,-11.276695,0.0
beta_time_drive,-6.444936,0.391347,-16.468579,0.0
beta_time_pt,-3.514548,0.258213,-13.611031,0.0
beta_time_walk,-8.205087,0.364391,-22.517272,0.0


In [677]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification
Number of estimated parameters,5,8
Sample size,5000,5000
Final log likelihood,-4566.446495,-4264.834219
Akaike Information Criterion,9142.892989,8545.668438
Bayesian Information Criterion,9175.478955,8597.805983
asc_cycle,-3.832002,-4.508888
asc_drive,-1.235559,-1.885804
asc_pt,-0.528693,-2.339783
beta_cost,-0.169777,-0.14607
beta_time,-5.45014,


### Cost specification

We compare with the model where we specify the price, to be sure to have the most interesting model for the future models. (walk and cycling don't have cost, so only 2 specific parameter for cost)

In [678]:
beta_cost_drive = Beta('beta_cost_drive', 0, None, None, 0)
beta_cost_pt = Beta('beta_cost_pt', 0, None, None, 0)

New utility function

In [679]:
v_walk_model1_spec_cost = beta_time * time_walk  
v_cycle_model1_spec_cost = asc_cycle + beta_time * time_cycle 
v_pt_model1_spec_cost = asc_pt + beta_time * time_pt + beta_cost_pt * cost_pt
v_drive_model1_spec_cost = asc_drive + beta_time * time_drive + beta_cost_drive * cost_drive

In [680]:
V_model1_spec_cost = {1: v_walk_model1_spec_cost , 2: v_cycle_model1_spec_cost, 3: v_pt_model1_spec_cost, 4: v_drive_model1_spec_cost}
logprob_model1_spec_cost = models.loglogit(V_model1_spec_cost, av, travel_mode)
biogeme_model1_spec_cost = bio.BIOGEME(database, logprob_model1_spec_cost)
biogeme_model1_spec_cost.modelName = 'Model_1_cost_specifiaction'
all_results['Model1_cost_specification'] = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.767069,0.107739,-34.964729,0.00000
asc_drive,-1.137434,0.080194,-14.183572,0.00000
asc_pt,-0.831988,0.064856,-12.828261,0.00000
beta_cost_drive,-0.214653,0.021273,-10.090326,0.00000
beta_cost_pt,0.061553,0.028466,2.162364,0.03059
beta_time,-5.411713,0.202914,-26.669951,0.00000


In [681]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification
Number of estimated parameters,5,8,6
Sample size,5000,5000,5000
Final log likelihood,-4566.446495,-4264.834219,-4509.365231
Akaike Information Criterion,9142.892989,8545.668438,9030.730461
Bayesian Information Criterion,9175.478955,8597.805983,9069.83362
asc_cycle,-3.832002,-4.508888,-3.767069
asc_drive,-1.235559,-1.885804,-1.137434
asc_pt,-0.528693,-2.339783,-0.831988
beta_cost,-0.169777,-0.14607,
beta_time,-5.45014,,-5.411713


### Comparaison

we use log-likehood ratio as the model 0 is a reduction of the model 1

In [682]:
old  = results_generic
new  = results_alt_spec_time
new.likelihood_ratio_test(old, 0.05) #level of the statistics for a level of significance of 5%?

LRTuple(message='H0 can be rejected at level 5.0%', statistic=603.2245515971099, threshold=7.814727903251179)

# Model 2

In [683]:
LPMC.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent'],
      dtype='object')

In [684]:
model_base=biogeme_model1

In [685]:
#we're adding the attribute distance to our model 
#we're adding the interaction between age and distance
pt_interchanges=Variable('pt_interchanges')
driving_license=Variable('driving_license')

In [686]:
#defining associated coefficients
beta_pt_inter= Beta('beta_pt_inter', 0, None, None, 0)
beta_license= Beta('beta_license', 0, None, None, 0)

In [687]:
#the specification, we only consider the alternative distance for walking
#the interaction distance age is considered just for public trasnport
v_walk_model2 = v_walk_model1
v_cycle_model2 = v_cycle_model1
v_pt_model2 = v_pt_model1 + beta_pt_inter*pt_interchanges
v_drive_model2 = v_drive_model1 + asc_drive*beta_license*driving_license

In [688]:
#the estimation results
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model_2'] = biogeme_model2.estimate()
results_model2 = biogeme_model2.estimate()
results_model2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.565996,0.196913,-23.187928,0.000000
asc_drive,-2.781829,0.138459,-20.091322,0.000000
asc_pt,-2.421344,0.131373,-18.431000,0.000000
beta_cost,-0.140456,0.013888,-10.113083,0.000000
beta_license,-0.517733,0.027956,-18.519837,0.000000
beta_pt_inter,-0.107643,0.089733,-1.199589,0.230299
beta_time_cycle,-6.304988,0.559452,-11.269930,0.000000
beta_time_drive,-7.003285,0.411117,-17.034762,0.000000
beta_time_pt,-3.540500,0.282483,-12.533479,0.000000
beta_time_walk,-8.374423,0.376903,-22.219063,0.000000


In [689]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2
Number of estimated parameters,5,8,6,10
Sample size,5000,5000,5000,5000
Final log likelihood,-4566.446495,-4264.834219,-4509.365231,-4039.286539
Akaike Information Criterion,9142.892989,8545.668438,9030.730461,8098.573078
Bayesian Information Criterion,9175.478955,8597.805983,9069.83362,8163.74501
asc_cycle,-3.832002,-4.508888,-3.767069,-4.565996
asc_drive,-1.235559,-1.885804,-1.137434,-2.781829
asc_pt,-0.528693,-2.339783,-0.831988,-2.421344
beta_cost,-0.169777,-0.14607,,-0.140456
beta_time,-5.45014,,-5.411713,


In [690]:
#we test base_model against model_2 where base_model is a restricted version of model_2 suing the looglikehood
#ratio test

In [691]:
#we test them using the null hypothesis that our base model is the true model

In [692]:
results_base=model_base.estimate()

In [693]:
results_model2.likelihood_ratio_test(results_base, 0.05) #level of the statistics for a level of significance of 5%?

LRTuple(message='H0 can be rejected at level 5.0%', statistic=451.09535982406396, threshold=5.991464547107979)

In [694]:
#then depending on if the value I obtained was higher or lower than a certain number, we reject or accept the null hypothesis

In [695]:
from scipy.stats import chi2
threshold = chi2.ppf(.95, 2)
threshold #just in case we might want to compare to this threshold

5.991464547107979

# Model 3

We will conduct a non-linear transformation of time

In [696]:
model_base=biogeme_model2
log_time_walk = database.DefineVariable('log_time_walk', log(time_walk))
log_time_cycle = database.DefineVariable('log_time_cycle', log(time_cycle))
log_time_pt = database.DefineVariable('log_time_pt', log(time_pt))
log_time_drive = database.DefineVariable('log_time_drive', log(time_drive))
log_beta_time_drive = Beta('log_beta_time_drive', 0, None, None, 0)
log_beta_time_pt = Beta('log_beta_time_pt', 0, None, None, 0)
log_beta_time_walk = Beta('log_beta_time_walk', 0, None, None, 0)
log_beta_time_cycle = Beta('log_beta_time_cycle', 0, None, None, 0)

In [697]:
v_walk_model3 = log_beta_time_walk * log_time_walk
v_cycle_model3 = asc_cycle + log_beta_time_cycle * log_time_cycle
v_pt_model3 = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model3 = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [698]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.440982,0.226833,-10.761128,0.000000
asc_drive,-0.681228,0.179498,-3.795183,0.000148
asc_pt,1.677112,0.147211,11.392588,0.000000
beta_cost,-0.137087,0.013205,-10.381271,0.000000
beta_license,-2.097700,0.533994,-3.928324,0.000086
beta_pt_inter,-0.093232,0.073972,-1.260361,0.207539
log_beta_time_cycle,-2.464811,0.131799,-18.701346,0.000000
log_beta_time_drive,-2.344575,0.101327,-23.138719,0.000000
log_beta_time_pt,-2.025552,0.120400,-16.823475,0.000000
log_beta_time_walk,-4.598905,0.133100,-34.552362,0.000000


In [699]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3
Number of estimated parameters,5,8,6,10,10
Sample size,5000,5000,5000,5000,5000
Final log likelihood,-4566.446495,-4264.834219,-4509.365231,-4039.286539,-3975.362113
Akaike Information Criterion,9142.892989,8545.668438,9030.730461,8098.573078,7970.724226
Bayesian Information Criterion,9175.478955,8597.805983,9069.83362,8163.74501,8035.896158
asc_cycle,-3.832002,-4.508888,-3.767069,-4.565996,-2.440982
asc_drive,-1.235559,-1.885804,-1.137434,-2.781829,-0.681228
asc_pt,-0.528693,-2.339783,-0.831988,-2.421344,1.677112
beta_cost,-0.169777,-0.14607,,-0.140456,-0.137087
beta_time,-5.45014,,-5.411713,,


We will compare Model_2 and Model_3 with a cox test, thus we will create a generic model

In [700]:
v_walk_model_generic = log_beta_time_walk * log_time_walk + beta_time_walk * time_walk
v_cycle_model_generic = asc_cycle + log_beta_time_cycle * log_time_cycle + beta_time_cycle * time_cycle
v_pt_model_generic = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges + beta_time_pt * time_pt
v_drive_model_generic = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license + beta_time_drive * time_drive

In [701]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.082704,0.680848,-3.058986,2.220874e-03
asc_drive,-0.303066,0.371968,-0.814764,4.152074e-01
asc_pt,2.509257,0.472751,5.307774,1.109724e-07
beta_cost,-0.129699,0.013881,-9.343950,0.000000e+00
beta_license,-4.760330,5.839478,-0.815198,4.149591e-01
beta_pt_inter,-0.035823,0.085237,-0.420274,6.742856e-01
beta_time_cycle,-2.585312,0.794433,-3.254287,1.136773e-03
beta_time_drive,-3.079869,0.578255,-5.326140,1.003220e-07
beta_time_pt,-2.947979,0.534669,-5.513654,3.514584e-08
beta_time_walk,-1.603892,0.469279,-3.417778,6.313465e-04


In [702]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic
Number of estimated parameters,5,8,6,10,10,14
Sample size,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446495,-4264.834219,-4509.365231,-4039.286539,-3975.362113,-3948.929663
Akaike Information Criterion,9142.892989,8545.668438,9030.730461,8098.573078,7970.724226,7925.859326
Bayesian Information Criterion,9175.478955,8597.805983,9069.83362,8163.74501,8035.896158,8017.100031
asc_cycle,-3.832002,-4.508888,-3.767069,-4.565996,-2.440982,-2.082703
asc_drive,-1.235559,-1.885804,-1.137434,-2.781829,-0.681228,-0.303066
asc_pt,-0.528693,-2.339783,-0.831988,-2.421344,1.677112,2.50926
beta_cost,-0.169777,-0.14607,,-0.140456,-0.137087,-0.129699
beta_time,-5.45014,,-5.411713,,,


In [703]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=180.7137520941542, threshold=9.487729036781154)

In [704]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=52.86490068871444, threshold=9.487729036781154)

In this case the cox-test tells us to find better models, we will thus use a box-cox transform

In [705]:
lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0)
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)
v_walk_model_boxcox = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_boxcox  = asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_boxcox  = asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model_boxcox  = asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [706]:
V_model_boxcox = {1: v_walk_model_boxcox , 2: v_cycle_model_boxcox, 3: v_pt_model_boxcox, 4: v_drive_model_boxcox}
logprob_model_boxcox = models.loglogit(V_model_boxcox, av, travel_mode)
biogeme_model_boxcox = bio.BIOGEME(database, logprob_model_boxcox)
biogeme_model_boxcox.modelName = 'Model_boxcox'
all_results['Model_boxcox'] = biogeme_model_boxcox.estimate()
results_model_boxcox = biogeme_model_boxcox.estimate()
results_model_boxcox.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.795061,0.287939,-9.707131,0.000000e+00
asc_drive,-1.326267,0.237232,-5.590590,2.262992e-08
asc_pt,1.642055,0.177951,9.227567,0.000000e+00
beta_cost,-0.129342,0.013805,-9.369446,0.000000e+00
beta_elapsed_cycle,-3.663212,0.287349,-12.748312,0.000000e+00
beta_elapsed_time_drive,-3.702159,0.275504,-13.437781,0.000000e+00
beta_elapsed_time_pt,-2.719102,0.190929,-14.241411,0.000000e+00
beta_elapsed_walk,-5.668601,0.237126,-23.905484,0.000000e+00
beta_license,-1.091348,0.190710,-5.722551,1.049363e-08
beta_pt_inter,-0.147479,0.080675,-1.828067,6.753944e-02


In [707]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox
Number of estimated parameters,5,8,6,10,10,14,11
Sample size,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446495,-4264.834219,-4509.365231,-4039.286539,-3975.362113,-3948.929663,-3953.048869
Akaike Information Criterion,9142.892989,8545.668438,9030.730461,8098.573078,7970.724226,7925.859326,7928.097739
Bayesian Information Criterion,9175.478955,8597.805983,9069.83362,8163.74501,8035.896158,8017.100031,7999.786864
asc_cycle,-3.832002,-4.508888,-3.767069,-4.565996,-2.440982,-2.082703,-2.795066
asc_drive,-1.235559,-1.885804,-1.137434,-2.781829,-0.681228,-0.303066,-1.326267
asc_pt,-0.528693,-2.339783,-0.831988,-2.421344,1.677112,2.50926,1.642057
beta_cost,-0.169777,-0.14607,,-0.140456,-0.137087,-0.129699,-0.129342
beta_time,-5.45014,,-5.411713,,,,


In [708]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_boxcox.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=172.47533924106028, threshold=3.841458820694124)

# Model 4

In [709]:

model_base=biogeme_model_boxcox
V_nested=V_model_boxcox
#create the nests
Mu_motorized=Beta("Mu_motorized",1,0,None,0)
Mu_notmotorized=Beta("Mu_notmotorized",1,0,None,0)
motorized= Mu_motorized,[3,4]
notmotorized=Mu_notmotorized, [1,2]
nests_motor=motorized, notmotorized


In [710]:
logprob_model_nested = models.lognested(V_nested, av, nests_motor, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'Model_nested_motor'
all_results['Model_nested_motor'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_motorized,1.575774,0.087260,18.058320,0.000000e+00
Mu_notmotorized,0.930458,0.118267,7.867442,3.552714e-15
asc_cycle,-2.113435,0.245607,-8.604953,0.000000e+00
asc_drive,-0.039553,0.005691,-6.949968,3.653744e-12
asc_pt,2.128084,0.114768,18.542533,0.000000e+00
beta_cost,-0.073193,0.011018,-6.643198,3.069500e-11
beta_elapsed_cycle,-2.595400,0.232769,-11.150131,0.000000e+00
beta_elapsed_time_drive,-2.573957,0.180606,-14.251766,0.000000e+00
beta_elapsed_time_pt,-1.936470,0.129158,-14.992993,0.000000e+00
beta_elapsed_walk,-4.783138,0.259072,-18.462550,0.000000e+00


This model is rejected as Mu_notmotorized is inferior to 1. We will try a crossnested model between motored and private.

In [711]:
#define the crossnests
Mu_private=Beta('Mu_private',3,1,None,0)
Mu_motorcross=Beta('Mu_Motorcross',3,1,None,0)
Alpha_Motor=Beta('Alpha_Motor',0.5,0,1,0)
Alpha_Private=1-Alpha_Motor

alpha_motor={1:0.0,
             2:0.0,
             3:Alpha_Motor,
             4:1.0 }
alpha_private={1:1.0,
               2:1.0,
               3:Alpha_Private,
               4:0.0}

nest_motor=Mu_motorcross, alpha_motor
nest_private=Mu_private, alpha_private
cross_nests=nest_private, nest_motor

In [712]:
logprob_model_crossnested = models.logcnl_avail(V_nested, av, cross_nests, travel_mode)
logprob_model_crossnested.changeInitValues(results_model_nested.getBetaValues())
biogeme_model_crossnested = bio.BIOGEME(database, logprob_model_crossnested)
biogeme_model_crossnested.modelName = 'Model_crossnested'
results_model_crossnested = biogeme_model_crossnested.estimate()
all_results['Model_crossnested'] = results_model_crossnested
results_model_crossnested.getEstimatedParameters()

,Value,Active bound,Rob. Std err,Rob. t-test,Rob. p-value
Alpha_Motor,1.000000,1.0,0.000071,14121.369989,0.000000e+00
Mu_Motorcross,1.595423,0.0,0.089640,17.798052,0.000000e+00
Mu_private,1.000000,1.0,0.135607,7.374277,1.652012e-13
asc_cycle,-2.190943,0.0,0.254119,-8.621705,0.000000e+00
asc_drive,-0.039433,0.0,0.005702,-6.916290,4.636291e-12
asc_pt,2.095873,0.0,0.115540,18.139813,0.000000e+00
beta_cost,-0.072404,0.0,0.010914,-6.633759,3.272427e-11
beta_elapsed_cycle,-2.673115,0.0,0.233895,-11.428719,0.000000e+00
beta_elapsed_time_drive,-2.523236,0.0,0.181341,-13.914339,0.000000e+00
beta_elapsed_time_pt,-1.902849,0.0,0.129605,-14.681928,0.000000e+00


The model is also rejected as alpha is 1. Moreover we see that mu_private is 1. We try another simple nest with the use of a vehicule:

In [713]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=1,[1]
nests_vehicle=vehicle, not_vehicle


In [714]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_vehicle,1.470052,0.073316,20.050998,0.000000e+00
asc_cycle,-0.873310,0.189203,-4.615737,3.917033e-06
asc_drive,-0.040997,0.009316,-4.400882,1.078116e-05
asc_pt,2.127870,0.103075,20.643936,0.000000e+00
beta_cost,-0.082929,0.011376,-7.290154,3.095302e-13
beta_elapsed_cycle,-2.478297,0.181123,-13.682970,0.000000e+00
beta_elapsed_time_drive,-2.587218,0.155056,-16.685679,0.000000e+00
beta_elapsed_time_pt,-1.942603,0.113064,-17.181415,0.000000e+00
beta_elapsed_walk,-4.800232,0.216575,-22.164264,0.000000e+00
beta_license,-26.125101,5.845915,-4.468950,7.860464e-06


In [715]:
#t-test H0:true value is 1
mu = table_nestedvehicle.loc['Mu_vehicle', 'Value']
mu_stderr = table_nestedvehicle.loc['Mu_vehicle', 'Rob. Std err']
tested_value = 1
ttest = (mu - tested_value) / mu_stderr
ttest

6.411346458573096

In [716]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox,Model_nested_motor,Model_crossnested,Model_nested_vehicle
Number of estimated parameters,5,8,6,10,10,14,11,13,14,12
Sample size,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446495,-4264.834219,-4509.365231,-4039.286539,-3975.362113,-3948.929663,-3953.048869,-3927.320927,-3927.449424,-3939.743937
Akaike Information Criterion,9142.892989,8545.668438,9030.730461,8098.573078,7970.724226,7925.859326,7928.097739,7880.641854,7882.898847,7903.487873
Bayesian Information Criterion,9175.478955,8597.805983,9069.83362,8163.74501,8035.896158,8017.100031,7999.786864,7965.365365,7974.139552,7981.694191
asc_cycle,-3.832002,-4.508888,-3.767069,-4.565996,-2.440982,-2.082703,-2.795066,-2.113435,-2.190943,-0.87331
asc_drive,-1.235559,-1.885804,-1.137434,-2.781829,-0.681228,-0.303066,-1.326267,-0.039553,-0.039433,-0.040997
asc_pt,-0.528693,-2.339783,-0.831988,-2.421344,1.677112,2.50926,1.642057,2.128084,2.095873,2.12787
beta_cost,-0.169777,-0.14607,,-0.140456,-0.137087,-0.129699,-0.129342,-0.073193,-0.072404,-0.082929
beta_time,-5.45014,,-5.411713,,,,,,,


In [717]:
results_base=model_base.estimate()
results_model_nestedvehicle.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=26.609865091666506, threshold=3.841458820694124)

In [718]:
V_pref=V_nested
nest_pref=nests_vehicle
model_name_pref='Model_nested_vehicle'
result_pref=results_model_nestedvehicle
estimation_pref=table_nestedvehicle

### Market share

In [719]:
LPMC.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent,log_time_walk,log_time_cycle,log_time_pt,log_time_drive
0,18,4,0,0,3,3,6,5,0.0,1,...,0,0.196389,0.0,0.53,0.0,0.035361,-0.271153,-1.462456,-0.681548,-1.627658
1,20,5,1,0,4,3,1,5,0.0,1,...,0,0.117222,0.0,0.41,0.0,0.097156,-0.513607,-1.696449,-0.812181,-2.143684
2,21,5,1,1,4,3,1,5,0.0,1,...,0,0.167778,0.0,0.46,0.0,0.243377,-0.505746,-1.586101,-0.974185,-1.785115
3,22,6,0,0,3,3,6,1,1.0,1,...,0,0.093889,1.5,0.23,0.0,0.204142,-1.072295,-2.061820,-1.416754,-2.365643
4,31,8,1,5,1,3,1,1,1.0,1,...,0,0.055000,1.5,0.14,0.0,0.075758,-1.749339,-2.905485,-1.972083,-2.900422


In [720]:
LPMC[(LPMC["household_id"]==5) & (LPMC["person_n"]==1)].shape[0]

2

Some individuals make several trips. To account for this in the computation of the weights, we'll consider an individual making n trips as n indivudals. 

In [721]:
census = {
    'male_41_more':  1633263,
    'male_40_less':  2676249,
    'female_41_more':   1765143,
    'female_40_less':  2599058,
}

In [722]:
total = sum(census.values())
total

8673713

In [723]:
filters = {
    'male_41_more': (LPMC.age >= 41) & (LPMC.female == 0),
    'male_40_less': (LPMC.age <= 40) & (LPMC.female == 0),
    'female_41_more': (LPMC.age >= 41) & (LPMC.female == 1),
    'female_40_less': (LPMC.age <= 40) & (LPMC.female == 1),
}

In [724]:
sample_segments = {
    k: v.sum() for k, v in filters.items()
}
sample_segments

{'male_41_more': 1149,
 'male_40_less': 1275,
 'female_41_more': 1196,
 'female_40_less': 1380}

In [725]:
total_sample = sum(sample_segments.values())
total_sample

5000

In [726]:
weights = {
    k: census[k] * total_sample / (v * total) 
    for k, v in sample_segments.items()
}
weights

{'male_41_more': 0.8194096069112643,
 'male_40_less': 1.2099886308951033,
 'female_41_more': 0.8507729467060965,
 'female_40_less': 1.085680009425514}

In [727]:
for k, f in filters.items():
    LPMC.loc[f, 'weights'] = weights[k] 

In [728]:
#we check the weights sum to the sample size
sum_weights = LPMC['weights'].sum()
sum_weights

5000.0

# Market Shares


We will there use the market_share function defined in lab session 03-indicators, but we adapted it to our models


Here we DO NOT simulate weights ? is it ok ?

## Market Shares with Modelpref

In [729]:
female=Variable('female')
age=Variable('age')
weights=Variable('weights')

In [730]:
type(weights)
IndicatorTuple = namedtuple('IndicatorTuple', 'value lower upper') 

In [731]:
def market_share(utility):
    """Calculate the market shares of all alternatives, given the
    specification of the utility functions.

    :param utility: specification of the utility functions. It is a
        dict where the keys are the IDs of the alternatives, and the
        values are the expressions of the utility functions.
    :type utility: dict(int: biogeme.expressions.Expression)

    :return: a dictionary where each entry corresponds to an
        alternative, and associates its name with the InticatorTuple
        containing the value of the market share, and the lower and
        upper bounds of the 90% confidence interval.
    :rtype: dict(str: IndicatorTuple)

    """
    prob_WALK = models.lognested(utility, av, nests_vehicle, 1)
    prob_CYCLE = models.lognested(utility, av, nests_vehicle, 2)
    prob_PT = models.lognested(utility, av, nests_vehicle, 3)
    prob_DRIVE = models.lognested(utility, av, nests_vehicle, 4)
    """
    prob_WALK = models.loglogit(utility, None, 1)
    prob_CYCLE= models.loglogit(utility, None, 2)
    prob_PT = models.loglogit(utility, None, 3)
    prob_DRIVE = models.loglogit(utility, None, 4)
    """

    simulate = {
        'Prob. walk': prob_WALK,
        'Prob. cycle': prob_CYCLE,
        'Prob. PT': prob_PT,
        'Prob. drive': prob_DRIVE,
        'weights' : weights
        
    }
    biosim = bio.BIOGEME(database, simulate)
    simulated_values = biosim.simulate(results_model_nestedvehicle.getBetaValues())

    # We also calculate confidence intervals for the calculated quantities

    betas = biogeme_model_nestedvehicle.freeBetaNames()
    b = results_model_nestedvehicle.getBetasForSensitivityAnalysis(betas,useBootstrap=False)
    left, right = biosim.confidenceIntervals(b, 0.9)

    # Market shares are calculated using the weighted mean of the individual probabilities
    #Since lognested outputs the logarithm of the wanted probability, we'll have to compose the occurences
    #of the proba by the exponential

    ## Alternative WALK

    simulated_values['Weighted prob. walk'] = simulated_values['weights'] * np.exp(simulated_values['Prob. walk'])
    left['Weighted prob. walk'] = left['weights'] * np.exp(left['Prob. walk'])
    right['Weighted prob. walk'] = right['weights'] * np.exp(right['Prob. walk'])

    marketShare_walk = simulated_values['Weighted prob. walk'].mean()
    marketShare_walk_left = left['Weighted prob. walk'].mean()
    marketShare_walk_right = right['Weighted prob. walk'].mean()
    
    
    ## Alternative cycle

    simulated_values['Weighted prob. cycle'] =simulated_values['weights'] * np.exp(simulated_values['Prob. cycle'])
    left['Weighted prob. cycle'] = left['weights'] * np.exp(left['Prob. cycle'])
    right['Weighted prob. cycle'] = right['weights'] * np.exp(right['Prob. cycle'])

    marketShare_cycle = simulated_values['Weighted prob. cycle'].mean()
    marketShare_cycle_left = left['Weighted prob. cycle'].mean()
    marketShare_cycle_right = right['Weighted prob. cycle'].mean()

    ## Alternative pt

    simulated_values['Weighted prob. PT'] = (
        simulated_values['weights'] * np.exp(simulated_values['Prob. PT'])
    )
    left['Weighted prob. PT'] = left['weights'] * np.exp(left['Prob. PT'])
    right['Weighted prob. PT'] = right['weights'] * np.exp(right['Prob. PT'])

    marketShare_PT = simulated_values['Weighted prob. PT'].mean()
    marketShare_PT_left = left['Weighted prob. PT'].mean()
    marketShare_PT_right = right['Weighted prob. PT'].mean()
    
                                                         
    ## Alternative drive

    simulated_values['Weighted prob. drive'] = (
        simulated_values['weights'] * np.exp(simulated_values['Prob. drive'])
    )
    left['Weighted prob. drive'] = left['weights'] * np.exp(left['Prob. drive'])
    right['Weighted prob. drive'] = right['weights'] * np.exp(right['Prob. drive'])

    marketShare_drive = simulated_values['Weighted prob. drive'].mean()
    marketShare_drive_left = left['Weighted prob. drive'].mean()
    marketShare_drive_right = right['Weighted prob. drive'].mean()

    return {
            'Walk': IndicatorTuple(
                value=marketShare_walk, 
                lower=marketShare_walk_left, 
                upper=marketShare_walk_right
            ),
            'Cycle': IndicatorTuple(
                value=marketShare_cycle, 
                lower=marketShare_cycle_left, 
                upper=marketShare_cycle_right
            ),
            'Public transportation': IndicatorTuple(
                value=marketShare_PT, 
                lower=marketShare_PT_left, 
                upper=marketShare_PT_right
            ),
            'Drive': IndicatorTuple(
                value=marketShare_drive, 
                lower=marketShare_drive_left, 
                upper=marketShare_drive_right
            )
    }

In [732]:
ms = market_share(V_nested)
for k, v in ms.items():
    print(
        f'Market share for {k}: {100*v.value:.2f}% '
        f'[{100*v.lower:.2f}%, '
        f'{100*v.upper:.2f}%]'
    )

#we check it sums up to 100
total=0
for k, v in ms.items():
    total+=100*v.value
print('Total : ', total)

Market share for Walk: 17.26% [16.28%, 18.72%]
Market share for Cycle: 2.98% [2.47%, 3.81%]
Market share for Public transportation: 35.66% [33.77%, 39.33%]
Market share for Drive: 44.10% [39.37%, 46.09%]
Total :  100.0


In [733]:
count_walk=0
count_cycle=0
count_pt=0
count_drive=0
for c in LPMC['travel_mode']:
    if c==1:
        count_walk+=1
    if c==2:
        count_cycle+=1
    if c==3:
        count_pt+=1
    if c==4:
        count_drive+=1
print('Market share for walk ' +  str(count_walk/5000))
print('Market share for cycle ' +  str(count_cycle/5000))   
print('Market share for pt ' +  str(count_pt/5000))   
print('Market share for drive ' +  str(count_drive/5000))   

Market share for walk 0.1714
Market share for cycle 0.0294
Market share for pt 0.3498
Market share for drive 0.4494


##   Forecasting

Scenario 1 : an increase of car cost by 15%.  We use the beta elapsed and lambda as defined in model 3. 

Only the utility function for the car changes. The others take the values of Model_pref

In [734]:
v_walk_scenario1 = V_nested[1]
v_cycle_scenario1   = V_nested[2]
v_pt_scenario1   = V_nested[3]
v_drive_scenario1   = asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive * (1+0.15) + asc_drive*beta_license*driving_license


V_scenario1  = {1: v_walk_scenario1  , 2: v_cycle_scenario1 , 3: v_pt_scenario1 , 4: v_drive_scenario1 }

In [735]:
ms1 = market_share(V_scenario1)
for k, v in ms1.items():
    print(
        f'Market share for {k}: {100*v.value:.2f}% '
        f'[{100*v.lower:.2f}%, '
        f'{100*v.upper:.2f}%]'
    )


#we check it sums up to 100
total=0
for k, v in ms1.items():
    total+=100*v.value
print('Total : ', total)

Market share for Walk: 17.29% [16.40%, 18.56%]
Market share for Cycle: 3.01% [2.48%, 3.78%]
Market share for Public transportation: 36.02% [34.25%, 39.25%]
Market share for Drive: 43.69% [39.68%, 45.62%]
Total :  100.0


Scenario 2 : Decrease of public transport cost by 15%

In [736]:
v_walk_scenario2 = V_nested[1]
v_cycle_scenario2   = V_nested[2]
v_pt_scenario2   = asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt * (1-0.15) + beta_pt_inter*pt_interchanges
v_drive_scenario2   = V_nested[4]


V_scenario2  = {1: v_walk_scenario2  , 2: v_cycle_scenario2 , 3: v_pt_scenario2 , 4: v_drive_scenario2 }

In [737]:
ms2 = market_share(V_scenario2)
for k, v in ms2.items():
    print(
        f'Market share for {k}: {100*v.value:.2f}% '
        f'[{100*v.lower:.2f}%, '
        f'{100*v.upper:.2f}%]'
    )

#we check it sums up to 100
total=0
for k, v in ms2.items():
    total+=100*v.value
print('Total : ', total)

Market share for Walk: 17.22% [16.20%, 18.57%]
Market share for Cycle: 2.95% [2.55%, 3.80%]
Market share for Public transportation: 36.09% [34.23%, 40.01%]
Market share for Drive: 43.74% [38.90%, 45.68%]
Total :  100.0


Question 3 : public transportation total revenue

In [738]:
def revenues(cost, utility):
    """Calculate the revenues of all alternatives, given the
    specification of the utility functions and the expression of the cost.

    :param cost: expression to calculate the cost of the public transportation.
    :type cost: biogeme.expressions.Expression

    :param utility: specification of the utility functions. It is a
        dict where the keys are the IDs of the alternatives, and the
        values are the expressions of the utility functions.
    :type utility: dict(int: biogeme.expressions.Expression)
    
    :return: tuple containing the value of the revenues, as well as
        the lower and upper bound of the 90% confidence interval.
    :rtype: IndicatorTuple    
    """
    prob_WALK = models.lognested(utility, av, nests_vehicle, 1)
    prob_CYCLE = models.lognested(utility, av, nests_vehicle, 2)
    prob_PT = models.lognested(utility, av, nests_vehicle, 3)
    prob_DRIVE = models.lognested(utility, av, nests_vehicle, 4)

    simulate = {
        'weight': weights,
        'Revenues PT': -prob_PT * cost,
    }
    biosim = bio.BIOGEME(database, simulate)
    simulated_values = biosim.simulate(results_model_nestedvehicle.getBetaValues())

    # We also calculate confidence intervals for the calculated quantities

    betas = biogeme_model_nestedvehicle.freeBetaNames()
    b = results_model_nestedvehicle.getBetasForSensitivityAnalysis(betas,useBootstrap=False)
    left, right = biosim.confidenceIntervals(b, 0.9)

    # Revenues are calculated using the weighted mean of the individual quantities


    simulated_values['Weighted revenues PT'] = (
        simulated_values['weight'] * simulated_values['Revenues PT']
    )
    left['Weighted revenues PT'] = left['weight'] * left['Revenues PT']
    right['Weighted revenues PT'] = right['weight'] * right['Revenues PT']

    revenues_PT = simulated_values['Weighted revenues PT'].mean()
    revenues_PT_left = left['Weighted revenues PT'].mean()
    revenues_PT_right = right['Weighted revenues PT'].mean()


    return IndicatorTuple(
        value=revenues_PT, 
        lower=revenues_PT_left, 
        upper=revenues_PT_right
    )

Total Revenues without any changement

In [739]:
revenue_0 = revenues(cost_pt,V_nested)
print(
        f'Revenue for Scenario 0: {revenue_0[0]:.3f} '
        f'[{revenue_0[1]:.2f}, '
        f'{revenue_0[2]:.2f}]'
    )

Revenue for Scenario 0: 1.936 [1.67, 2.05]


Scenario 1 : an increase of car cost by 15%, the factor is 1 as the price for pt doen't change

In [740]:
cost_1=cost_pt*1
revenue_scenario_1 = revenues(cost_1,V_scenario1)
print(
        f'Revenue for Scenario 1: {revenue_scenario_1[0]:.3f} '
        f'[{revenue_scenario_1[1]:.2f}, '
        f'{revenue_scenario_1[2]:.2f}]'
    )


Revenue for Scenario 1: 1.916 [1.62, 2.04]


Scenario 2 : a decrease of public transport cost by 15%, the factor is 0.85

In [744]:
cost_2=cost_pt*0.85
revenue_scenario_2 = revenues(cost_2,V_scenario2)
print(
        f'Revenue for the Scenario 2: {revenue_scenario_2[0]:.2f} '
        f'[{revenue_scenario_2[1]:.2f}, '
        f'{revenue_scenario_2[2]:.2f}]'
    )

Revenue for the Scenario 2: 1.61 [1.44, 1.70]


Question 4 : the average value of time for car and public transportation.

Average value of time for car

In [742]:
count_walk=0
count_cycle=0
count_pt=0
count_drive=0
for c in LPMC['travel_mode']:
    if c==1:
        count_walk+=1
    if c==2:
        count_cycle+=1
    if c==3:
        count_pt+=1
    if c==4:
        count_drive+=1
print('Market share for walk ' +  str(count_walk/5000))
print('Market share for cycle ' +  str(count_cycle/5000))   
print('Market share for pt ' +  str(count_pt/5000))   
print('Market share for drive ' +  str(count_drive/5000))  

Market share for walk 0.1714
Market share for cycle 0.0294
Market share for pt 0.3498
Market share for drive 0.4494


In [743]:
type(estimation_pref.Value)

pandas.core.series.Series